In [1]:
# get openai api key from openai_config.json file
import json
with open('../openai_config.json') as f:
    openai_api_key = json.load(f)['openai_api_key']

In [2]:
%%capture

%env OPENAI_API_KEY = {openai_api_key}

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
llm.invoke("Hello, how are you?").content

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [5]:
from langchain.document_loaders import CSVLoader
input_loader_kor = CSVLoader(r"../../guidance_for_environmental_impact_factor_mapping_on_aws/assets/input/sample_coupang_product_names_groceries_kor.csv", encoding='utf-8-sig')
input_doc_kor = input_loader_kor.load()

input_doc_kor[0].page_content

'품목명: 오뚜기 흰밥, 210g, 24개'

In [6]:
from langchain_core.prompts import PromptTemplate

prompt = """너는 식료품을 HS 코드로 분류하는 전과정영향평가(Life Cycle Analysis, LCA) 전문가야.

나는 환경 확장 투입 산출(Environmentally Extended Input Output, EEIO) 환경 영향 계수(Environmental Impact Factors, EIF)를 기반으로 한 식료품의 전과정영향평가를 수행하고 싶어. 나는 식료품의 재료 및 제조 단계와 관련된 환경적 영향에 관심 있어.

나는 주어진 물품에 가장 적합한 HS 코드와 항목명을 알고 싶어. 선택에 대한 근거를 함께 설명해야 해.
출력 형식은 JSON으로, 키는 "가장_적합한_HS_코드", "가장_적합한_HS_항목명", "판단_근거"야.

물품:
{product_name}

다음 중 제공된 물품과 가장 적합한 HS 코드는 무엇이지?

주어진 정보를 최대한 활용해. 정보가 제한되어 있다고 말하거나 추가 정보를 요청하지 마.
가장 적합한 코드와 항목명 선택해야 해. 선택에 대한 근거를 반드시 포함해야 해.
주어진 지침이나 정보를 반복하지 마.
전과정영향평가에 필요한 정보가 충분하지 않다고 말하지 마.
다른 말 없이 JSON 출력으로만 응답해."""
prompt = PromptTemplate.from_template(prompt)

In [8]:
# Initialize results storage
results_kor = []

# Process each row using the graph
for doc in input_doc_kor:
    # # Extract CommodityDescription from the page_content using regex
    # match = re.search(r"CommodityDescription: (.+)", doc.page_content)
    # activity = match.group(1).strip() if match else None  # Extract the description

    activity = doc.page_content # Extract the Product Name

    if activity:
        msg = prompt.invoke({"product_name": activity})
        result = llm.invoke(msg)
        
        # Extract context and answer from the state after processing
        results_kor.append(result.content)

# Print the results for debugging
print("Final results:", results_kor)

Final results: ['```json\n{\n    "가장_적합한_HS_코드": "1904.90",\n    "가장_적합한_HS_항목명": "기타 조제 식료품",\n    "판단_근거": "오뚜기 흰밥은 즉석 조리된 쌀밥 제품으로, 주로 쌀을 주재료로 하여 가공된 식품이다. HS 코드 1904.90은 쌀을 포함한 다양한 곡물을 가공한 형태의 조제 식료품에 해당하므로, 이 제품에 적합하다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1902.30",\n    "가장_적합한_HS_항목명": "기타 파스타(조리된 것인지 또는 조리되지 않은 것인지, 또는 속을 채운 것인지에 상관없으며, 스파게티, 마카로니, 국수 등)",\n    "판단_근거": "신라면은 조리된 국수 제품이며, HS 코드 1902.30은 조리 여부와 상관없이 파스타 제품을 포괄하는 코드이다. 따라서 신라면과 같은 인스턴트 라면 제품은 이 코드에 포함될 수 있다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1602.49",\n    "가장_적합한_HS_항목명": "기타 돼지고기 제품",\n    "판단_근거": "스팸은 가공된 돼지고기 제품으로, HS 코드 1602.49는 기타 가공된 돼지고기 제품을 포함합니다. \'스팸 25% 라이트\'는 전형적인 가공육 제품으로, 이 범주에 적합합니다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1604.14",\n    "가장_적합한_HS_항목명": "기타 어류의 조제품(고추장에 절인 것 포함)",\n    "판단_근거": "동원 고추 참치는 참치 조제품으로, 1604.14는 어류의 조제품 중에서 특히 고추장과 같은 양념에 절인 제품을 포함하는 코드로 가장 적합하다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "2201.90",\n    "가장_적합한_HS_항목명": "기타의 물",\n    "판단_근거": 

In [47]:
results_kor = ['```json\n{\n    "가장_적합한_HS_코드": "1904.90",\n    "가장_적합한_HS_항목명": "기타 조제 식료품",\n    "판단_근거": "오뚜기 흰밥은 즉석 조리된 쌀밥 제품으로, 주로 쌀을 주재료로 하여 가공된 식품이다. HS 코드 1904.90은 쌀을 포함한 다양한 곡물을 가공한 형태의 조제 식료품에 해당하므로, 이 제품에 적합하다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1902.30",\n    "가장_적합한_HS_항목명": "기타 파스타(조리된 것인지 또는 조리되지 않은 것인지, 또는 속을 채운 것인지에 상관없으며, 스파게티, 마카로니, 국수 등)",\n    "판단_근거": "신라면은 조리된 국수 제품이며, HS 코드 1902.30은 조리 여부와 상관없이 파스타 제품을 포괄하는 코드이다. 따라서 신라면과 같은 인스턴트 라면 제품은 이 코드에 포함될 수 있다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1602.49",\n    "가장_적합한_HS_항목명": "기타 돼지고기 제품",\n    "판단_근거": "스팸은 가공된 돼지고기 제품으로, HS 코드 1602.49는 기타 가공된 돼지고기 제품을 포함합니다. \'스팸 25% 라이트\'는 전형적인 가공육 제품으로, 이 범주에 적합합니다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1604.14",\n    "가장_적합한_HS_항목명": "기타 어류의 조제품(고추장에 절인 것 포함)",\n    "판단_근거": "동원 고추 참치는 참치 조제품으로, 1604.14는 어류의 조제품 중에서 특히 고추장과 같은 양념에 절인 제품을 포함하는 코드로 가장 적합하다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "2201.90",\n    "가장_적합한_HS_항목명": "기타의 물",\n    "판단_근거": "탐사수 무라벨은 포장된 물로, HS 코드 2201.90은 일반적으로 포장된 물, 미네랄 워터 및 기타 비알콜 음료에 사용됩니다. 2L 용량의 물이 24개로 포장되어 있어 \'기타의 물\' 범주에 속하는 것이 가장 적합합니다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1602.32",\n    "가장_적합한_HS_항목명": "기타 조제한 것(냉동 여부 상관 없음)",\n    "판단_근거": "품목인 \'곰곰 더 진한 갈비탕 (냉동)\'은 가공된 고기 제품으로, 주로 갈비와 같은 육류를 포함하고 있으며, 조리 및 가공 과정을 거쳐 냉동된 형태로 제공된다. HS 코드 1602.32는 조제 또는 보존한 육류(냉동 여부 상관 없음)에 해당하므로, 이 제품에 가장 적합한 HS 코드로 판단된다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "2202.99",\n    "가장_적합한_HS_항목명": "기타 비알콜 음료",\n    "판단_근거": "블루다이아몬드 아몬드 브리즈 오리지널은 아몬드 우유로 분류되며, 이는 비알콜성 음료에 속합니다. HS 코드 2202.99는 \'기타 비알콜 음료\'를 포괄하므로 해당 제품에 적합합니다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "1902.20",\n    "가장_적합한_HS_항목명": "냉동 만두",\n    "판단_근거": "비비고 왕교자는 냉동상태의 조리된 식품으로, 주로 밀가루와 고기, 야채 등 다양한 재료로 만든 만두 제품이다. HS 코드 1902.20은 \'냉동 만두\'로, 냉동 상태의 가공 식품에 적합하다. 이 제품의 특성과 일치하여 가장 적합한 분류로 판단된다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "0805.10",\n    "가장_적합한_HS_항목명": "Fresh mandarins (including tangerines and satsumas)",\n    "판단_근거": "주어진 물품은 \'감귤\'이며, 이는 일반적으로 귤 또는 만다린 오렌지로 분류됩니다. HS 코드 0805.10은 신선한 만다린(탠저린 및 사츠마 포함)에 해당하며, 이는 제공된 물품 설명과 일치합니다. \'2kg\'이라는 중량 정보와 \'로열과\'라는 품질 분류는 HS 코드 선택에 영향을 미치지 않습니다."\n}\n```', '```json\n{\n    "가장_적합한_HS_코드": "210690",\n    "가장_적합한_HS_항목명": "기타 조제식료품",\n    "판단_근거": "담터 호두 아몬드 율무차는 다양한 재료가 혼합된 형태의 즉석 차 제품으로, HS 코드 210690은 다양한 형태의 조제식료품을 포함하며, 특정한 범주에 명시되지 않은 혼합 식품에 적합하다."\n}\n```']
len(results_kor)
results_kor[0]

'```json\n{\n    "가장_적합한_HS_코드": "1904.90",\n    "가장_적합한_HS_항목명": "기타 조제 식료품",\n    "판단_근거": "오뚜기 흰밥은 즉석 조리된 쌀밥 제품으로, 주로 쌀을 주재료로 하여 가공된 식품이다. HS 코드 1904.90은 쌀을 포함한 다양한 곡물을 가공한 형태의 조제 식료품에 해당하므로, 이 제품에 적합하다."\n}\n```'

In [16]:
# Save results to a CSV file
import csv
# from datetime import datetime

# Get the current date and time
# now = datetime.now()
# timestamp = now.strftime("%Y%m%d_%H%M%S")  # Format: YYYYMMDD_HHMMSS

# Create a unique file name with date and time!!
file_name_kor = f"coupang_openai_gt_kor.csv"

with open(file_name_kor, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(results_kor)

print("Results saved to {file_name_kor}")

Results saved to {file_name_kor}


In [48]:
for i in range(len(results_kor)):
    results_kor[i] = json.loads(results_kor[i].strip("```").lstrip("json").replace("\n", "").replace(" ", ""))
    print(results_kor[i])

{'가장_적합한_HS_코드': '1904.90', '가장_적합한_HS_항목명': '기타조제식료품', '판단_근거': '오뚜기흰밥은즉석조리된쌀밥제품으로,주로쌀을주재료로하여가공된식품이다.HS코드1904.90은쌀을포함한다양한곡물을가공한형태의조제식료품에해당하므로,이제품에적합하다.'}
{'가장_적합한_HS_코드': '1902.30', '가장_적합한_HS_항목명': '기타파스타(조리된것인지또는조리되지않은것인지,또는속을채운것인지에상관없으며,스파게티,마카로니,국수등)', '판단_근거': '신라면은조리된국수제품이며,HS코드1902.30은조리여부와상관없이파스타제품을포괄하는코드이다.따라서신라면과같은인스턴트라면제품은이코드에포함될수있다.'}
{'가장_적합한_HS_코드': '1602.49', '가장_적합한_HS_항목명': '기타돼지고기제품', '판단_근거': "스팸은가공된돼지고기제품으로,HS코드1602.49는기타가공된돼지고기제품을포함합니다.'스팸25%라이트'는전형적인가공육제품으로,이범주에적합합니다."}
{'가장_적합한_HS_코드': '1604.14', '가장_적합한_HS_항목명': '기타어류의조제품(고추장에절인것포함)', '판단_근거': '동원고추참치는참치조제품으로,1604.14는어류의조제품중에서특히고추장과같은양념에절인제품을포함하는코드로가장적합하다.'}
{'가장_적합한_HS_코드': '2201.90', '가장_적합한_HS_항목명': '기타의물', '판단_근거': "탐사수무라벨은포장된물로,HS코드2201.90은일반적으로포장된물,미네랄워터및기타비알콜음료에사용됩니다.2L용량의물이24개로포장되어있어'기타의물'범주에속하는것이가장적합합니다."}
{'가장_적합한_HS_코드': '1602.32', '가장_적합한_HS_항목명': '기타조제한것(냉동여부상관없음)', '판단_근거': "품목인'곰곰더진한갈비탕(냉동)'은가공된고기제품으로,주로갈비와같은육류를포함하고있으며,조리및가공과정을거쳐냉동된형태로제공된다.HS코드1602.32는조제또는보존한육류(냉동여부상관없음)에해당하므로,이제품에가장적합한HS코드로판단된다.

In [49]:
for i in range(len(results_kor)):
    results_kor[i]["가장_적합한_HS_코드"] = results_kor[i].get("가장_적합한_HS_코드").replace(".", "").rjust(6, "0")

In [50]:
for result in results_kor:
    print(result)

{'가장_적합한_HS_코드': '190490', '가장_적합한_HS_항목명': '기타조제식료품', '판단_근거': '오뚜기흰밥은즉석조리된쌀밥제품으로,주로쌀을주재료로하여가공된식품이다.HS코드1904.90은쌀을포함한다양한곡물을가공한형태의조제식료품에해당하므로,이제품에적합하다.'}
{'가장_적합한_HS_코드': '190230', '가장_적합한_HS_항목명': '기타파스타(조리된것인지또는조리되지않은것인지,또는속을채운것인지에상관없으며,스파게티,마카로니,국수등)', '판단_근거': '신라면은조리된국수제품이며,HS코드1902.30은조리여부와상관없이파스타제품을포괄하는코드이다.따라서신라면과같은인스턴트라면제품은이코드에포함될수있다.'}
{'가장_적합한_HS_코드': '160249', '가장_적합한_HS_항목명': '기타돼지고기제품', '판단_근거': "스팸은가공된돼지고기제품으로,HS코드1602.49는기타가공된돼지고기제품을포함합니다.'스팸25%라이트'는전형적인가공육제품으로,이범주에적합합니다."}
{'가장_적합한_HS_코드': '160414', '가장_적합한_HS_항목명': '기타어류의조제품(고추장에절인것포함)', '판단_근거': '동원고추참치는참치조제품으로,1604.14는어류의조제품중에서특히고추장과같은양념에절인제품을포함하는코드로가장적합하다.'}
{'가장_적합한_HS_코드': '220190', '가장_적합한_HS_항목명': '기타의물', '판단_근거': "탐사수무라벨은포장된물로,HS코드2201.90은일반적으로포장된물,미네랄워터및기타비알콜음료에사용됩니다.2L용량의물이24개로포장되어있어'기타의물'범주에속하는것이가장적합합니다."}
{'가장_적합한_HS_코드': '160232', '가장_적합한_HS_항목명': '기타조제한것(냉동여부상관없음)', '판단_근거': "품목인'곰곰더진한갈비탕(냉동)'은가공된고기제품으로,주로갈비와같은육류를포함하고있으며,조리및가공과정을거쳐냉동된형태로제공된다.HS코드1602.32는조제또는보존한육류(냉동여부상관없음)에해당하므로,이제품에가장적합한HS코드로판단된다."}
{'가

In [56]:
to_csv = results_kor
keys = to_csv[0].keys()

with open(file_name_kor, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(to_csv)